<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Mathematics Basics

**With `NumPy`**

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Prime Numbers

### Pure Python

In [ ]:
!git clone https://github.com/tpq-classes/mathematics_basics.git
import sys
sys.path.append('mathematics_basics')


In [ ]:
def is_prime(I):
    if I % 2 == 0: return False  
    for i in range(3, int(I ** 0.5) + 1, 2):  
        if I % i == 0: return False  
    return True  

In [ ]:
n = int(1e8 + 3)  
n

In [ ]:
%time is_prime(n)

In [ ]:
p1 = int(1e8 + 7)  
p1

In [ ]:
%time is_prime(p1)

In [ ]:
p2 = 100109100129162907  

In [ ]:
p2.bit_length()  

In [ ]:
%time is_prime(p2)

### Numba

In [ ]:
import numba

In [ ]:
is_prime_nb = numba.jit(is_prime)

In [ ]:
%time is_prime_nb(n)  

In [ ]:
%time is_prime_nb(n)  

In [ ]:
%time is_prime_nb(p1)

In [ ]:
%time is_prime_nb(p2)  

### Cython

In [ ]:
%load_ext Cython

In [ ]:
%%cython
def is_prime_cy1(I):
    if I % 2 == 0: return False
    for i in range(3, int(I ** 0.5) + 1, 2):
        if I % i == 0: return False
    return True

In [ ]:
%timeit is_prime(p1)

In [ ]:
%timeit is_prime_cy1(p1)

In [ ]:
%%cython
def is_prime_cy2(long I):  
    cdef long i  
    if I % 2 == 0: return False
    for i in range(3, int(I ** 0.5) + 1, 2):
        if I % i == 0: return False
    return True

In [ ]:
%timeit is_prime_cy2(p1)

In [ ]:
%time is_prime_nb(p2)

In [ ]:
%time is_prime_cy2(p2)

### Multiprocessing

<b style="color: red;">This currently does not work on Mac M1 machines.</b>

In [ ]:
import multiprocessing as mp

In [ ]:
pool = mp.Pool(processes=4)  

In [ ]:
%time pool.map(is_prime, 10 * [p1])  

In [ ]:
%time pool.map(is_prime_nb, 10 * [p2])  

In [ ]:
%time pool.map(is_prime_cy2, 10 * [p2])  

## Monte Carlo Simulation

### Python

In [ ]:
import math
import numpy as np

In [ ]:
S0 = 36.
T = 1.0
r = 0.06
sigma = 0.2

In [ ]:
M = 100  
I = 50000  

In [ ]:
def mcs_simulation_py(p):
    M, I = p
    dt = T / M
    S = np.zeros((M + 1, I))
    S[0] = S0
    rn = np.random.standard_normal(S.shape)  
    for t in range(1, M + 1):  
        for i in range(I):  
            S[t, i] = S[t - 1, i] * math.exp((r - sigma ** 2 / 2) * dt +
                                         sigma * math.sqrt(dt) * rn[t, i])  
    return S      

In [ ]:
%time S = mcs_simulation_py((M, I))

In [ ]:
S[-1].mean()  

In [ ]:
S0 * math.exp(r * T)  

### NumPy

In [ ]:
def mcs_simulation_np(p):
    M, I = p
    dt = T / M
    S = np.zeros((M + 1, I))
    S[0] = S0
    rn = np.random.standard_normal(S.shape)
    for t in range(1, M + 1):  
        S[t] = S[t - 1] * np.exp((r - sigma ** 2 / 2) * dt +
                               sigma * math.sqrt(dt) * rn[t]) 
    return S      

In [ ]:
%time S = mcs_simulation_np((M, I))

In [ ]:
S[-1].mean()

In [ ]:
%timeit S = mcs_simulation_np((M, I))

### Numba 

In [ ]:
mcs_simulation_nb = numba.jit(mcs_simulation_py)

In [ ]:
%time S = mcs_simulation_nb((M, I))  

In [ ]:
%time S = mcs_simulation_nb((M, I))  

In [ ]:
S[-1].mean()

In [ ]:
%timeit S = mcs_simulation_nb((M, I))  

### Cython &mdash; Sequential

In [ ]:
%%cython
import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport exp, sqrt
cdef float S0 = 36.
cdef float T = 1.0
cdef float r = 0.06
cdef float sigma = 0.2
@cython.boundscheck(False)
@cython.wraparound(False)
def mcs_simulation_cy(p):
    cdef int M, I
    M, I = p
    cdef int t, i
    cdef float dt = T / M
    cdef double[:, :] S = np.zeros((M + 1, I))
    cdef double[:, :] rn = np.random.standard_normal((M + 1, I))
    S[0] = S0
    for t in range(1, M + 1):
        for i in range(I):
            S[t, i] = S[t - 1, i] * exp((r - sigma ** 2 / 2) * dt +
                                         sigma * sqrt(dt) * rn[t, i])
    return np.array(S) 

In [ ]:
%time S = mcs_simulation_cy((M, I))

In [ ]:
S[-1].mean()

In [ ]:
%timeit S = mcs_simulation_cy((M, I))

### Multiprocessing

<b style="color: red;">This currently does not work on Mac M1 machines.</b>


In [ ]:
import multiprocessing as mp

In [ ]:
pool = mp.Pool(processes=4)  

In [ ]:
p = 20  

In [ ]:
%timeit S = np.hstack(pool.map(mcs_simulation_np, p * [(M, int(I / p))]))

In [ ]:
%timeit S = np.hstack(pool.map(mcs_simulation_nb, p * [(M, int(I / p))]))

In [ ]:
%timeit S = np.hstack(pool.map(mcs_simulation_cy, p * [(M, int(I / p))]))

<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>